In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/lankenau/isotools/src')

In [3]:
STRICT = False

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import math

path = '/project/hfa_work/ENCODE/data'
alignment_path = 'alignment_v45'
genome_file = 'GRCh38.p14.genome.fa'
genome_path = os.path.join(path, 'gencode_human/version_45', genome_file)

In [5]:
metadata_file = 'reads/metadata_tissue.tsv'
metadata = pd.read_csv(os.path.join(path, metadata_file), sep='\t')
metadata

,sample ID,organism,individual,sex,age,developmental stage,disease,group,Biosample term name,Description,Biosample summary,file,file size
0,ENCSR700EBI,Homo sapiens,W61,female,41 years,adult,normal,aorta,aorta,W61 aorta ENC4_88,Homo sapiens aorta tissue female adult (41 years),/project/hfa_work/yalan/golong/encode_tissue_d...,616290149
1,ENCSR425HFS,Homo sapiens,W73,female,59 years,adult,normal,aorta,aorta,ENC4_103 W73 aorta,Homo sapiens aorta tissue female adult (59 years),/project/hfa_work/yalan/golong/encode_tissue_d...,1168326040
2,ENCSR463IDK,Homo sapiens,E5194210,female,79 years,adult,normal,brain,dorsolateral prefrontal cortex,RUSH brain E5194210,Homo sapiens dorsolateral prefrontal cortex ti...,/project/hfa_work/yalan/golong/encode_tissue_d...,1145642918
3,ENCSR205QMF,Homo sapiens,E5115600,female,88 years,adult,normal,brain,dorsolateral prefrontal cortex,RUSH brain E5115600,Homo sapiens dorsolateral prefrontal cortex ti...,/project/hfa_work/yalan/golong/encode_tissue_d...,452782844
4,ENCSR169YNI,Homo sapiens,E7461192,female,90 or above years,adult,normal,brain,dorsolateral prefrontal cortex,RUSH brain E7461192,Homo sapiens dorsolateral prefrontal cortex ti...,/project/hfa_work/yalan/golong/encode_tissue_d...,866488131
5,ENCSR094NFM,Homo sapiens,E4368365,female,90 or above years,adult,normal,brain,dorsolateral prefrontal cortex,RUSH brain E4368365,Homo sapiens dorsolateral prefrontal cortex ti...,/project/hfa_work/yalan/golong/encode_tissue_d...,193641719
6,ENCSR997RFW,Homo sapiens,W76,male,40 years,adult,normal,colon,mucosa of descending colon,ENC4_77 W76 mucosa of descending colon,Homo sapiens mucosa of descending colon tissue...,/project/hfa_work/yalan/golong/encode_tissue_d...,800980029
7,ENCSR450GAR,Homo sapiens,W73,female,59 years,adult,normal,colon,left colon,W73 left colon ENC4_87,Homo sapiens left colon tissue female adult (5...,/project/hfa_work/yalan/golong/encode_tissue_d...,730236367
8,ENCSR984OAE,Homo sapiens,W76,male,40 years,adult,normal,heart,heart right ventricle,ENC4_52 W76 heart right ventricle,Homo sapiens heart right ventricle tissue male...,/project/hfa_work/yalan/golong/encode_tissue_d...,686400256
9,ENCSR994YZY,Homo sapiens,W76,male,40 years,adult,normal,heart,heart left ventricle,ENC4_53 W76 heart left ventricle,Homo sapiens heart left ventricle tissue male ...,/project/hfa_work/yalan/golong/encode_tissue_d...,658616334


In [6]:
import logging
from isotools import Transcriptome
from isotools import __version__ as isotools_version
# set up logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)
logger=logging.getLogger('isotools')
logger.info(f'This is isotools version {isotools_version}')

INFO:This is isotools version 0.3.5rc10


In [7]:
annotation_file=os.path.join(path, 'gencode_human/version_45', 'gencode.v45.chr_patch_hapl_scaff.annotation_sorted.gff3.gz')
#create the IsoTools transcriptome object from the reference annotation
isoseq=Transcriptome.from_reference(annotation_file)

INFO:importing reference from gff3 file /project/hfa_work/ENCODE/data/gencode_human/version_45/gencode.v45.chr_patch_hapl_scaff.annotation_sorted.gff3.gz
  0%|          | 0.00/81.7M [00:00<?, ?B/s]

100%|█████████▉| 81.7M/81.7M [00:46<00:00, 1.84MB/s]
INFO:skipped the following categories: {'CDS', 'three_prime_UTR', 'five_prime_UTR', 'stop_codon_redefined_as_selenocysteine'}


In [8]:
for i, row in metadata.iterrows():
    sample_name = row['sample ID']
    # file is the full (wrong) path, we just need the filename without the extension
    sample_file = os.path.join(path, alignment_path, row['file'].split('/')[-1].split('.')[0] + '_aligned.bam')
    if not os.path.exists(sample_file):
        logger.error(f'File {sample_file} does not exist')
        continue
    group = row['group']
    isoseq.add_sample_from_bam(fn=sample_file, sample_name=sample_name, group=group, strictness=20 if STRICT else math.inf)
isoseq.sample_table

INFO:adding sample ENCSR700EBI from file /project/hfa_work/ENCODE/data/alignment_v45/ENCFF144KHH_aligned.bam
  0%|          | 0.00/2.19M [00:00<?, ?reads/s, chr=chr1]

100%|██████████| 2.19M/2.19M [02:55<00:00, 12.5kreads/s, chr=KI270757.1]
INFO:skipped 2720 reads aligned fraction of less than 0.75.
INFO:skipped 950420 secondary alignments (0x100), alignment that failed quality check (0x200) or PCR duplicates (0x400)
INFO:ignoring 123726 chimeric alignments with less than 2 reads
INFO:imported 1106210 nonchimeric reads (including  1377 chained chimeric alignments) and 9024 chimeric reads with coverage of at least 2.
INFO:adding sample ENCSR425HFS from file /project/hfa_work/ENCODE/data/alignment_v45/ENCFF902BIU_aligned.bam
100%|██████████| 2.74M/2.74M [05:05<00:00, 8.96kreads/s, chr=KI270757.1]
INFO:skipped 9838 reads aligned fraction of less than 0.75.
INFO:skipped 667992 secondary alignments (0x100), alignment that failed quality check (0x200) or PCR duplicates (0x400)
INFO:ignoring 50134 chimeric alignments with less than 2 reads
INFO:imported 2013267 nonchimeric reads (including  5697 chained chimeric alignments) and 2918 chimeric reads with cove

,name,file,group,nonchimeric_reads,chimeric_reads
0,ENCSR700EBI,/project/hfa_work/ENCODE/data/alignment_v45/EN...,aorta,1106210,9024
1,ENCSR425HFS,/project/hfa_work/ENCODE/data/alignment_v45/EN...,aorta,2013267,2918
2,ENCSR463IDK,/project/hfa_work/ENCODE/data/alignment_v45/EN...,brain,2035784,18875
3,ENCSR205QMF,/project/hfa_work/ENCODE/data/alignment_v45/EN...,brain,963304,8488
4,ENCSR169YNI,/project/hfa_work/ENCODE/data/alignment_v45/EN...,brain,1593059,11852
5,ENCSR094NFM,/project/hfa_work/ENCODE/data/alignment_v45/EN...,brain,501838,5193
6,ENCSR997RFW,/project/hfa_work/ENCODE/data/alignment_v45/EN...,colon,1272092,16637
7,ENCSR450GAR,/project/hfa_work/ENCODE/data/alignment_v45/EN...,colon,1282606,3025
8,ENCSR984OAE,/project/hfa_work/ENCODE/data/alignment_v45/EN...,heart,1053851,9911
9,ENCSR994YZY,/project/hfa_work/ENCODE/data/alignment_v45/EN...,heart,992724,11205


In [9]:
# compute qc metrics
isoseq.add_qc_metrics(genome_path)
# add ORF predictions
isoseq.add_orf_prediction(genome_path)

100%|██████████| 541983/541983 [29:24<00:00, 307.23genes/s]


In [12]:
suffix = '_strict' if STRICT else ''

In [14]:
isoseq.save(f'results/isoseq_v45{suffix}.pkl')

INFO:saving transcriptome to results/isoseq_v45.pkl


In [16]:
isoseq.write_gtf(f'results/isoseq_v45{suffix}.gtf',
                 min_coverage=5, gzip=False, query="")

INFO:writing gtf file to results/isoseq_v45.gtf


In [17]:
# export transcript table with the same filter criteria:
transcript_tab=isoseq.transcript_table( groups=isoseq.groups(),tpm=True,coverage=True,
                                       min_coverage=5, progress_bar=True,
                                       query="")
# write to csv file
transcript_tab.to_csv(f'results/demonstration_dataset_substantial_transcripts{suffix}.csv',
                      index=False, sep='\t')

transcript_tab.head()

100%|██████████| 541983/541983 [00:20<00:00, 26557.64genes/s]


,chr,transcript_start,transcript_end,strand,gene_id,gene_name,transcript_nr,transcript_length,num_exons,exon_starts,...,ovary_sum_coverage,vessel_sum_coverage,aorta_sum_tpm,brain_sum_tpm,colon_sum_tpm,heart_sum_tpm,lung_sum_tpm,muscle_sum_tpm,ovary_sum_tpm,vessel_sum_tpm
0,GL000194.1,61662,62921,-,PB_novel_188538,PB_novel_188538,0,1259,1,61662,...,2,4,0.320567,0.58893,0.391436,0.771620,0.444486,0.00000,0.584430,2.083491
1,GL000194.1,53591,55442,-,ENSG00000277400.1,ENSG00000277400,0,1851,1,53591,...,3,3,2.243966,0.19631,1.957179,1.043957,1.926105,0.40793,0.876645,1.562618
2,GL000194.1,53591,115065,-,ENSG00000277400.1,ENSG00000277400,1,2224,3,"53591,112791,114985",...,13,3,3.526232,0.58893,1.957179,1.361683,2.370591,0.40793,3.798797,1.562618
3,GL000194.1,53591,55503,-,ENSG00000277400.1,ENSG00000277400,2,1127,2,"53591,54677",...,0,0,0.320567,0.19631,0.391436,0.226947,0.148162,0.40793,0.000000,0.000000
4,GL000194.1,53591,115065,-,ENSG00000277400.1,ENSG00000277400,3,1439,4,"53591,54677,112791,114985",...,3,1,0.641133,0.39262,0.391436,0.090779,0.148162,0.00000,0.876645,0.520873
